### Краткая формулировка задачи: необходимо по слову определить к какой части речи оно относится и найти его начальную форму.

### Итоговое решение: с помощью SVM определяю к какой части речи относится слово. Далее, ищу в обучающей выборке однокоренные слова и отвечаю ответом для однокоренного слова. 
### Результат сразу получился достаточно хороший, поэтому много попыток улучшить его я не предпринимал. Все свелось к тому, что я искал, по какой части слова лучше искать однокоренное слово в обучающей выборке (для этого, конечно, нужно было заметить, что слова часто повторяются, меняется только форма, поэтому легко найти однокоренные).

In [130]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RandomizedLogisticRegression
from sklearn.metrics import mean_squared_error, roc_auc_score
import sklearn.linear_model
from sklearn.base import BaseEstimator
from sklearn import svm
import numpy as np
import math
import matplotlib.pyplot as plt
import collections
from sklearn.pipeline import Pipeline, make_pipeline
from scipy import optimize
from sklearn import datasets, cross_validation, metrics, neighbors
from matplotlib.colors import ListedColormap
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [8]:
sample_submission = pd.read_csv("task2_lemmas_sample_submission")
test = pd.read_csv("task2_lemmas_test")
train_data = []
train_target = []
with open("task2_lemmas_train") as file:
    for line in file:
        string = line.split(',')
        train_data.append(string[1])
        words = []
        for i in range(2, len(string)):
            cur_str = string[i].split('\n')[0]
            word = cur_str.split('+')
            words.append(word)
        train_target.append(words)
train_data = train_data[1:]
train_target = train_target[1:]

In [9]:
test_str = []
with open("task2_lemmas_test") as file:
    for line in file:
        string = line.split('\n')[0]
        string_2 = string.split(',')
        test_str.append(string_2[1])
test_str = test_str[1:]

In [10]:
train_target_for_test = []
for i in range(len(train_target)):
    train_target_for_test.append(train_target[i][0][1])

In [11]:
ind_of_many_ans = []
for i in range(len(train_target)):
    if len(train_target[i]) > 1:
        ind_of_many_ans.append(i)

In [12]:
ind_of_one_ans = []
for i in range(len(train_target)):
    if i not in ind_of_many_ans:
        ind_of_one_ans.append(i)

In [13]:
train_data_one = []
train_target_one_word = []
train_target_one_class = []
for i in ind_of_one_ans:
    train_data_one.append(train_data[i])
    train_target_one_word.append(train_target[i][0][0])
    train_target_one_class.append(train_target[i][0][1])

In [14]:
A = 0
N = 0
V = 0
for i in range(len(train_target_one_class)):
    if train_target_one_class[i] == 'A':
        A += 1
    if train_target_one_class[i] == 'V':
        V += 1
    if train_target_one_class[i] == 'N':
        N += 1

In [15]:
train_V = []
train_N = []
train_A = []
train_target_V = []
train_target_N = []
train_target_A = []
for i in range(len(train_data)):
    if train_target_for_test[i] == 'V':
        train_V.append(train_data[i])
        train_target_V.append(train_target[i][0][0])
    if train_target_for_test[i] == 'N':
        train_N.append(train_data[i])
        train_target_N.append(train_target[i][0][0])
    if train_target_for_test[i] == 'A':
        train_A.append(train_data[i])
        train_target_A.append(train_target[i][0][0])

In [16]:
print("A =", A, "N =", N, "V =", V)

A = 12316 N = 8963 V = 95198


## Кросс-валидация и всякие попытки улучшить результат:
### (Итоговое решение ниже)

In [18]:
count_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(4, 6), lowercase=True, decode_error='ignore')
count_vectorizer.fit(train_data_one)
sparse_feature_matrix = count_vectorizer.transform(train_data_one)
model = svm.LinearSVC()
arr_model = cross_validation.cross_val_score(model, sparse_feature_matrix, 
                                                                 train_target_one_class, cv=5, scoring='accuracy')
print("model", np.mean(arr_model))

model 0.980150629337


In [19]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
   [train_data[i] for i in ind_of_one_ans], [train_target[i] for i in ind_of_one_ans], test_size=0.2, random_state=1)

In [20]:
comp_y_test = []
for i in range(len(y_test)):
    comp_y_test.append(np.array(y_test)[:,0,0][i] + "+" + np.array(y_test)[:,0,1][i])

In [21]:
y_train_word = []
y_test_word = []
y_train_class = []
y_test_class = []
for i in range(len(y_train)):
    y_train_word.append(y_train[i][0][0])
    y_train_class.append(y_train[i][0][1])
for i in range(len(y_test)):
    y_test_word.append(y_test[i][0][0])
    y_test_class.append(y_test[i][0][1])

In [22]:
X_train_N = []
X_train_V = []
X_train_A = []
X_test_N = []
X_test_V = []
X_test_A = []
y_train_word_N = []
y_train_word_V = []
y_train_word_A = []
y_test_word_N = []
y_test_word_V = []
y_test_word_A = []
for i in range(len(y_train_class)):
    if y_train_class[i] == 'N':
        X_train_N.append(X_train[i])
        y_train_word_N.append(y_train_word[i])
    if y_train_class[i] == 'V':
        X_train_V.append(X_train[i])
        y_train_word_V.append(y_train_word[i])
    if y_train_class[i] == 'A':
        X_train_A.append(X_train[i])
        y_train_word_A.append(y_train_word[i])
for i in range(len(y_test_class)):
    if y_test_class[i] == 'N':
        X_test_N.append(X_test[i])
        y_test_word_N.append(y_test_word[i])
    if y_test_class[i] == 'V':
        X_test_V.append(X_test[i])
        y_test_word_V.append(y_test_word[i])
    if y_test_class[i] == 'A':
        X_test_A.append(X_test[i])
        y_test_word_A.append(y_test_word[i])

In [75]:
ans = [0.0 for i in range(len(y_test_word_V))]
for i in range(len(X_test_V)):
    t = [-2.0 for l in range(5)]
    for j in range(len(X_train_V)):
        l = len(X_test_V[i])
        if l > 0:
            f_0 = X_train_V[j].find(X_test_V[i])
            f = X_train_V[j].find(X_test_V[i][:-1])
            f_2 = X_train_V[j].find(X_test_V[i][:-2])
            f_3 = X_train_V[j].find(X_test_V[i][:-3])
            f_4 = X_train_V[j].find(X_test_V[i][:round(l/2)])
        else:
            f = X_train_V[j].find(X_test_V[i][:int(l/2) + 2])
        if f_0 == 0:
            t[0] = j
        if f == 0:
            t[1] = j
        if f_2 == 0:
            t[2] = j
        if f_3 == 0:
            t[3] = j
        if f_4 == 0:
            t[4] = j
    for h in range(5):
        if t[h] != -2:
            ans[i] = y_train_word_V[t[h]]
            break
    if ans[i] == 0:
        ans[i] = X_test_V[i]

In [76]:
len([i for i in range(len(ans)) if ans[i] == 0.0])

0

In [77]:
metrics.accuracy_score(y_test_word_V, ans)

0.9958679847272347

In [99]:
ans_2 = [0.0 for i in range(len(y_test_word_N))]
for i in range(len(X_test_N)):
    t = [-2.0 for l in range(3)]
    for j in range(len(X_train_N)):
        l = len(X_test_N[i])
        if l - 1 > 0:
            f_0 = X_train_N[j].find(X_test_N[i])
            f = X_train_N[j].find(X_test_N[i][:-1])
            f_2 = X_train_N[j].find(X_test_N[i][:round(l/2)+4])
            #f_3 = X_train_N[j].find(X_test_N[i][:-3])
        else:
            f = X_train_N[j].find(X_test_N[i][:round(l/2)])
        if f_0 == 0:
            t[0] = j
        if f == 0:
            t[1] = j
        if f_2 == 0:
            t[2] = j
        #if f_3 == 0:
            #t[3] = j
    for h in range(3):
        if t[h] != -2:
            ans_2[i] = y_train_word_N[t[h]]
            break
    if ans_2[i] == 0:
        ans_2[i] = X_test_N[i]

In [100]:
metrics.accuracy_score(y_test_word_N, ans_2)

0.7332205301748449

In [48]:
len([i for i in range(len(ans_2)) if ans_2[i] == 0.0])

0

In [60]:
letter = []
for i in range(len(y_test_word_N)):
    letter.append(y_test_word_N[i][-3:])
letter = collections.Counter(letter)

In [113]:
ans_3 = [0.0 for i in range(len(y_test_word_A))]
for i in range(len(X_test_A)):
    t = [-2.0 for l in range(4)]
    for j in range(len(X_train_A)):
        l = len(X_test_A[i])
        if l - 1 > 0:
            f_0 = X_train_A[j].find(X_test_A[i])
            f = X_train_A[j].find(X_test_A[i][:-1])
            f_2 = X_train_A[j].find(X_test_A[i][:-2])
            f_3 = X_train_A[j].find(X_test_A[i][:round(l/2)+3])
        else:
            f = X_train_A[j].find(X_test_A[i][:round(l/2)])
        if f_0 == 0:
            t[0] = j
        if f == 0:
            t[1] = j
        if f_2 == 0:
            t[2] = j
        if f_3 == 0:
            t[3] = j
    for h in range(4):
        if t[h] != -2:
            ans_3[i] = y_train_word_A[t[h]]
            break
    if ans_3[i] == 0:
        ans_3[i] = X_test_A[i]

In [114]:
metrics.accuracy_score(y_test_word_A, ans_3)

0.8032445923460898

In [68]:
len([i for i in range(len(ans_3)) if ans_3[i] == 0.0])

0

## Далее следует итоговое решение:

In [115]:
count_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(4, 6), lowercase=False, decode_error='ignore')
count_vectorizer.fit(train_data_one)
sparse_feature_matrix = count_vectorizer.transform(train_data)
sparse_feature_matrix_test = count_vectorizer.transform(test_str)
model = svm.LinearSVC()
model.fit_transform(sparse_feature_matrix, train_target_for_test)
preds = model.predict(sparse_feature_matrix_test)

/home/evgeny/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [116]:
print(len(preds), len(test_str))

29661 29661


In [117]:
test_V = []
test_N = []
test_A = []
for i in range(len(preds)):
    if preds[i] == 'V':
        test_V.append(test_str[i])
    if preds[i] == 'N':
        test_N.append(test_str[i])
    if preds[i] == 'A':
        test_A.append(test_str[i])

In [118]:
ans_test = [0.0 for i in range(len(test_V))]
for i in range(len(test_V)):
    t = [-2.0 for l in range(5)]
    for j in range(len(train_V)):
        l = len(test_V[i])
        if l > 0:
            f_0 = train_V[j].find(test_V[i])
            f = train_V[j].find(test_V[i][:-1])
            f_2 = train_V[j].find(test_V[i][:-2])
            f_3 = train_V[j].find(test_V[i][:-3])
            f_4 = train_V[j].find(test_V[i][:round(l/2) + 2])
        else:
            f = train_V[j].find(test_V[i][:int(l/2) + 2])
        if f_0 == 0:
            t[0] = j
        if f == 0:
            t[1] = j
        if f_2 == 0:
            t[2] = j
        if f_3 == 0:
            t[3] = j
        if f_4 == 0:
            t[4] = j
    for h in range(5):
        if t[h] != -2:
            ans_test[i] = train_target_V[t[h]]
            break
    if ans_test[i] == 0:
        ans_test[i] = test_V[i]

In [119]:
total_answer = [0.0 for i in range(len(preds))]

In [120]:
k = 0
for i in range(len(preds)):
    if (preds[i] == 'V'):
        total_answer[i] = ans_test[k]
        k += 1

In [121]:
ans_test_2 = [0.0 for i in range(len(test_N))]
for i in range(len(test_N)):
    t = [-2.0 for l in range(3)]
    for j in range(len(train_N)):
        l = len(test_N[i])
        if l - 1 > 0:
            f_0 = train_N[j].find(test_N[i])
            f = train_N[j].find(test_N[i][:-1])
            f_2 = train_N[j].find(test_N[i][:round(l/2)+4])
            #f_3 = X_train_N[j].find(X_test_N[i][:-3])
        else:
            f = train_N[j].find(test_N[i][:round(l/2)])
        if f_0 == 0:
            t[0] = j
        if f == 0:
            t[1] = j
        if f_2 == 0:
            t[2] = j
        #if f_3 == 0:
            #t[3] = j
    for h in range(3):
        if t[h] != -2:
            ans_test_2[i] = train_target_N[t[h]]
            break
    if ans_test_2[i] == 0:
        ans_test_2[i] = test_N[i]

In [122]:
k = 0
for i in range(len(preds)):
    if (preds[i] == 'N'):
        total_answer[i] = ans_test_2[k]
        k += 1

In [123]:
ans_test_3 = [0.0 for i in range(len(test_A))]
for i in range(len(test_A)):
    t = [-2.0 for l in range(4)]
    for j in range(len(train_A)):
        l = len(test_A[i])
        if l - 1 > 0:
            f_0 = train_A[j].find(test_A[i])
            f = train_A[j].find(test_A[i][:-1])
            f_2 = train_A[j].find(test_A[i][:-2])
            f_3 = train_A[j].find(test_A[i][:round(l/2)+3])
        else:
            f = train_A[j].find(test_A[i][:round(l/2)])
        if f_0 == 0:
            t[0] = j
        if f == 0:
            t[1] = j
        if f_2 == 0:
            t[2] = j
        if f_3 == 0:
            t[3] = j
    for h in range(4):
        if t[h] != -2:
            ans_test_3[i] = train_target_A[t[h]]
            break
    if ans_test_3[i] == 0:
        ans_test_3[i] = test_A[i]

In [124]:
len([i for i in range(len(total_answer)) if total_answer[i] == 0.0])

3248

In [125]:
k = 0
for i in range(len(preds)):
    if (preds[i] == 'A'):
        total_answer[i] = ans_test_3[k]
        k += 1

In [126]:
len([i for i in range(len(total_answer)) if total_answer[i] == 0.0])

0

In [127]:
for i in range(len(total_answer)):
    if total_answer[i] != 0.0:
        total_answer[i] = total_answer[i] + "+" + preds[i]

In [128]:
sample_submission['Category'] = total_answer

In [129]:
sample_submission.to_csv("task2_linear_ans_3.tsv", sep=',', index=False)